# The Drivers of Errors in Single Unit Zestimates at Zillow
***

## Introduction

### Author
- Samuel Davila
 - Data Scientist
 - Zillow DS Department

### Purpose of project
- Identify what causes the accuracy of our estimates of property value ("Zestimates") to fluctuate. 

### Why is this project important?
- By understanding what improves and what weakens the accuracy of our estimates, we can shore up our weaknesses while maintaining our strengths. 

### Data source
- Single unit property data from Zillow table in Data Science Database
***

## Executive Summary

### Goals
- Improve original estimate of the log error by using clustering methodologies.

- Deliver the following:

    - zillow_clustering_project.ipynb

    - README.md

    - acquire.py

    - prep.py

    - preprocessing.py

    - model.py
    
    - A presentation that walks through each step of our project and this notebook as a whole.

### Analysis

Analysis goes here

### Recommendation

Recommendation goes here

### Expectation

Expectation goes here
***

## Project Planning

### Acquire

### Preparation

### Exploraton

### Model

### Conclusion
***

## Acquire
Acquire the data we need for our project from the zillow table in the data science database.
***

#### Importing modules needed for code in notebook to run.

In [6]:
# set up environment
from acquire import get_zillow_data

#### Using function in acquire.py file to import data then previewing data.

In [7]:
# create variable that will hold DF for easy access to data
df = get_zillow_data()

# previewing data
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,parcelid,logerror,transactiondate
0,2061546,11289917,1.0,None,None,2.0,3.0,NaN,6.0,2.0,...,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,11289917,-0.362001,2017-06-23
1,1834372,11705026,NaN,None,None,1.0,2.0,NaN,6.0,1.0,...,2016.0,23624.0,543.69,None,NaN,6.037232e+13,1772,11705026,-0.146056,2017-06-30
2,2121349,11389003,NaN,None,None,2.0,3.0,NaN,6.0,2.0,...,2016.0,449000.0,7673.19,None,NaN,6.037703e+13,3273,11389003,-0.325393,2017-06-01
3,2093710,11967869,NaN,None,None,1.0,2.0,NaN,5.0,1.0,...,2016.0,191821.0,3267.47,None,NaN,6.037185e+13,3429,11967869,-0.005566,2017-06-29
4,1288537,12035176,NaN,None,None,1.0,1.0,NaN,11.0,1.0,...,2016.0,146810.0,2926.19,None,NaN,6.037189e+13,5444,12035176,-0.114435,2017-05-24


### Acquire Takeaways
- Acquiring data from zillow database on the data science database server using the __get_zillow_data__ function
- Function needed to replicate this phase is located in the __acquire.py__ file

## Prepare
Reformat our acquired data so that it is ready for exploration via converting data types, handling null values, dealing with outliers, changing column names, and anything else necessary for our next steps.

Exact preparatory changes made are detailed in the cells below.

## Explore

## Modeling

## Overall Conclusion and Takeaways